# k Nearest Neighbor Classifier

## The overall video ratings for `Observer`(V1), `Strategy`(V2) and `MVC`(V3) patterns will be classified separately based on each learning style dimension scores below...
* A/R_Score
* S/I_Score
* Vi/Vb_Score
* S/G_Score

The classifiers were trained using `GridSearchCV`, a hyperparamater tuning technique, and the training set was split into `10-fold cross validation`. The average scores were collected after each classifier was 'train test split' a few times (various random states) with train-test.

***

# Importing the necessary libraries

In [89]:
### For data manipulation and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## To encode and scale the data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

### Machine Learning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE,RandomOverSampler

# Reading the dataset

In [90]:
pd.set_option('display.max_columns', None)
training = pd.read_csv('BDP_Cleaned.csv') #dataset after feature selection process for 5-Level Classification

# Dropping all irrelevant features
* `Time_Day_Watched` of all design patterns as features have already been extracted
* `Gender` as male and females are imbalanced and does not provide any useful insights
* `Duration` of each video as watch status and percentage duration have already been extracted from it

In [91]:
training.drop(['Time_Day_Watched_(V1-1)',
         'Time_Day_Watched_(V1-2)',
         'Time_Day_Watched_(V1-3)',
         'Time_Day_Watched_(V1-4)',
         'Time_Day_Watched_(V2-1)',
         'Time_Day_Watched_(V2-2)',
         'Time_Day_Watched_(V2-3)',
         'Time_Day_Watched_(V2-4)',
         'Time_Day_Watched_(V3-1)',
         'Time_Day_Watched_(V3-2)',
         'Time_Day_Watched_(V3-3)',
         'Time_Day_Watched_(V3-4)',
               'Gender',
               "V1-1_(1)",
               "V1-2_(1)",
               "V1-3_(1)",
               "V1-4_(1)",
               "V2-1_(2)",
               "V2-1_(2)",
               "V2-2_(2)",
               "V2-3_(2)",
               "V2-4_(2)",
               "V3-1_(3)",
               "V3-2_(3)",
               "V3-3_(3)",
               "V3-4_(3)",
               
        ], axis = 1,inplace = True)

# Getting numerical and categorical features for further tuning scaling and fitting

In [92]:
numerical_feats = training.dtypes[training.dtypes != "object"].index
categorical_feats = training.dtypes[training.dtypes == "object"].index

In [93]:
print("*"*100)
print(training[numerical_feats].columns)
print("*"*100)
print(training[categorical_feats].columns)
print("*"*100)
training.columns

****************************************************************************************************
Index(['V1_PercentWatched', 'V2_PercentWatched', 'V3_PercentWatched',
       'A/R_Score', 'Active', 'Reflective', 'S/I_Score', 'Sensing',
       'Intuitive', 'Vi/Vb_Score', 'Visual', 'Verbal', 'S/G_Score',
       'Sequential', 'Global', 'Quiz'],
      dtype='object')
****************************************************************************************************
Index(['CGPA_Class', 'Student_Rating_(V1-1)', 'Student_Rating_(V1-2)',
       'Student_Rating_(V1-3)', 'Student_Rating_(V1-4)', 'Overall_Rating_V1',
       'Watch_Status_V1', 'Student_Rating_(V2-1)', 'Student_Rating_(V2-2)',
       'Student_Rating_(V2-3)', 'Student_Rating_(V2-4)', 'Overall_Rating_V2',
       'Watch_Status_V2', 'Student_Rating_(V3-1)', 'Student_Rating_(V3-2)',
       'Student_Rating_(V3-3)', 'Student_Rating_(V3-4)', 'Overall_Rating_V3',
       'Watch_Status_V3', 'part_day_V1-1', 'part_day_V1-2', 'part_day_V1-

Index(['CGPA_Class', 'Student_Rating_(V1-1)', 'Student_Rating_(V1-2)',
       'Student_Rating_(V1-3)', 'Student_Rating_(V1-4)', 'Overall_Rating_V1',
       'Watch_Status_V1', 'V1_PercentWatched', 'Student_Rating_(V2-1)',
       'Student_Rating_(V2-2)', 'Student_Rating_(V2-3)',
       'Student_Rating_(V2-4)', 'Overall_Rating_V2', 'Watch_Status_V2',
       'V2_PercentWatched', 'Student_Rating_(V3-1)', 'Student_Rating_(V3-2)',
       'Student_Rating_(V3-3)', 'Student_Rating_(V3-4)', 'Overall_Rating_V3',
       'Watch_Status_V3', 'V3_PercentWatched', 'A/R_Score', 'Active',
       'Reflective', 'S/I_Score', 'Sensing', 'Intuitive', 'Vi/Vb_Score',
       'Visual', 'Verbal', 'S/G_Score', 'Sequential', 'Global', 'Quiz',
       'part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4',
       'part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4',
       'part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4'],
      dtype='object')

# Using MinMax Scaler to scale data from 0 to 1 since numerical features are not normally distributed

In [94]:
scaler = MinMaxScaler()
training['Quiz'] = scaler.fit_transform(training[['Quiz']])
training['A/R_Score'] = scaler.fit_transform(training[['A/R_Score']])
training['S/I_Score'] = scaler.fit_transform(training[['S/I_Score']])
training['Vi/Vb_Score'] = scaler.fit_transform(training[['Vi/Vb_Score']])
training['S/G_Score'] = scaler.fit_transform(training[['S/G_Score']])
training['V1_PercentWatched'] = scaler.fit_transform(training[['V1_PercentWatched']])
training['V2_PercentWatched'] = scaler.fit_transform(training[['V2_PercentWatched']])
training['V3_PercentWatched'] = scaler.fit_transform(training[['V3_PercentWatched']])
training.head()

,CGPA_Class,Student_Rating_(V1-1),Student_Rating_(V1-2),Student_Rating_(V1-3),Student_Rating_(V1-4),Overall_Rating_V1,Watch_Status_V1,V1_PercentWatched,Student_Rating_(V2-1),Student_Rating_(V2-2),Student_Rating_(V2-3),Student_Rating_(V2-4),Overall_Rating_V2,Watch_Status_V2,V2_PercentWatched,Student_Rating_(V3-1),Student_Rating_(V3-2),Student_Rating_(V3-3),Student_Rating_(V3-4),Overall_Rating_V3,Watch_Status_V3,V3_PercentWatched,A/R_Score,Active,Reflective,S/I_Score,Sensing,Intuitive,Vi/Vb_Score,Visual,Verbal,S/G_Score,Sequential,Global,Quiz,part_day_V1-1,part_day_V1-2,part_day_V1-3,part_day_V1-4,part_day_V2-1,part_day_V2-2,part_day_V2-3,part_day_V2-4,part_day_V3-1,part_day_V3-2,part_day_V3-3,part_day_V3-4
0,2.67 - 3.32,Excellent,Satisfactory,Very Good,Very Good,Very Good,Completed,0.747748,Very Good,Satisfactory,Very Good,Very Good,Very Good,Completed,0.440789,Excellent,Very Good,Very Good,Very Good,Very Good,Completed,0.632653,0.2,1,0,0.25,1,0,0.6,1,0,0.0,1,0,1.000,night,night,night,night,night,night,night,night,night,night,night,night
1,2.67 - 3.32,Very Good,Very Good,Very Good,Very Good,Very Good,Completed,0.747748,Very Good,Very Good,Very Good,Very Good,Very Good,Completed,0.440789,Very Good,Very Good,Very Good,Very Good,Very Good,Rewatched,0.653061,0.2,0,1,1.00,1,0,0.6,1,0,0.2,0,1,0.750,midnight,night,night,night,night,night,night,night,night,night,night,night
2,2.67 - 3.32,Excellent,Satisfactory,Very Good,Satisfactory,Very Good,Completed,0.747748,Very Good,Satisfactory,Very Good,Excellent,Very Good,Completed,0.440789,Excellent,Very Good,Very Good,Very Good,Very Good,Completed,0.632653,0.0,0,1,0.75,1,0,0.6,1,0,0.2,0,1,0.500,night,night,afternoon,afternoon,night,night,afternoon,afternoon,night,night,afternoon,afternoon
3,3.67 - 4.00,Excellent,Very Good,Excellent,Excellent,Excellent,Completed,0.747748,Excellent,Excellent,Very Good,Excellent,Excellent,Rewatched,0.618421,Excellent,Excellent,Very Good,Excellent,Excellent,Completed,0.632653,0.2,0,1,0.25,1,0,0.6,1,0,0.0,1,0,0.375,midnight,midnight,midnight,afternoon,midnight,midnight,midnight,midnight,midnight,midnight,midnight,midnight
4,2.67 - 3.32,Very Good,Very Good,Satisfactory,Very Good,Very Good,Completed,0.747748,Excellent,Very Good,Very Good,Satisfactory,Very Good,Completed,0.440789,Very Good,Very Good,Excellent,Very Good,Very Good,Completed,0.632653,0.2,1,0,0.00,1,0,0.4,1,0,0.2,0,1,0.750,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon,afternoon


# Encoding categorical values using label encoder

In [95]:
features =  ["CGPA_Class",
             "Overall_Rating_V1",
             "Overall_Rating_V2",
             "Overall_Rating_V3",
             "Watch_Status_V1",
             "Watch_Status_V2",
             "Watch_Status_V3",
             "part_day_V1-1",
             "part_day_V1-2",
             "part_day_V1-3",
             "part_day_V1-4",
             "part_day_V2-1",
             "part_day_V2-2",
             "part_day_V2-3",
             "part_day_V2-4",
             "part_day_V3-1",
             "part_day_V3-2",
             "part_day_V3-3",
             "part_day_V3-4"]

for i in features:
    training.loc[:,i] = LabelEncoder().fit_transform(training.loc[:,i]) 

### Overall_rating_V1 `(Observer)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [96]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'A/R_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_A = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_A

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.723577,0.701826,0.723577,0.702764,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.707317,0.706812,0.707317,0.689087,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.707317,0.692490,0.707317,0.697865,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
3,0.4,7,0.724490,0.713141,0.724490,0.704051,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.734694,0.740712,0.734694,0.725051,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.755102,0.765463,0.755102,0.736279,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
6,0.3,7,0.743243,0.707194,0.743243,0.711366,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
7,0.3,69,0.756757,0.762647,0.756757,0.741847,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.783784,0.818275,0.783784,0.769975,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.755102,0.773963,0.755102,0.733181,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [97]:
df_A.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.775510,0.783792,0.775510,0.756608
0.3,0.761261,0.762705,0.761261,0.741062
0.4,0.738095,0.739772,0.738095,0.721794
0.5,0.712737,0.700376,0.712737,0.696572


### Overall_rating_V1 `(Observer)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)MVC

In [98]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'S/I_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_B = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_B

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.756098,0.750183,0.756098,0.745078,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.682927,0.656884,0.682927,0.660219,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.707317,0.673710,0.707317,0.681894,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.785714,0.788661,0.785714,0.772102,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.734694,0.726553,0.734694,0.710562,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
5,0.4,101,0.704082,0.695862,0.704082,0.682032,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
6,0.3,7,0.783784,0.788632,0.783784,0.769625,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.783784,0.774535,0.783784,0.777216,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.770270,0.762056,0.770270,0.755782,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.755102,0.748299,0.755102,0.743696,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [99]:
df_B.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.789116,0.779241,0.789116,0.775293
0.3,0.779279,0.775074,0.779279,0.767541
0.4,0.741497,0.737025,0.741497,0.721565
0.5,0.715447,0.693592,0.715447,0.695730


### Overall_rating_V1 `(Observer)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [100]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'Vi/Vb_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_C = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_C

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.715447,0.702187,0.715447,0.705579,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.642276,0.636714,0.642276,0.612506,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.715447,0.705931,0.715447,0.699634,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.744898,0.748120,0.744898,0.744456,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.714286,0.724319,0.714286,0.699320,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.704082,0.684218,0.704082,0.676866,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.729730,0.707105,0.729730,0.709185,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.770270,0.782736,0.770270,0.751815,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.689189,0.688006,0.689189,0.665408,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.734694,0.716327,0.734694,0.702808,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."


In [101]:
df_C.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.775510,0.773405,0.775510,0.756879
0.3,0.729730,0.725949,0.729730,0.708803
0.4,0.721088,0.718886,0.721088,0.706881
0.5,0.691057,0.681611,0.691057,0.672573


### Overall_rating_V1 `(Observer)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [102]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V1']]
X = training[['CGPA_Class',
              'Watch_Status_V1',
              'S/G_Score','Quiz','part_day_V1-1', 'part_day_V1-2', 'part_day_V1-3', 'part_day_V1-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_D = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_D

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.691057,0.682401,0.691057,0.684869,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.609756,0.606585,0.609756,0.588900,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.747967,0.733133,0.747967,0.730586,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.704082,0.708455,0.704082,0.665841,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.724490,0.761199,0.724490,0.709135,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.673469,0.632250,0.673469,0.630589,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.702703,0.682370,0.702703,0.673542,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.702703,0.695714,0.702703,0.674472,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.729730,0.735011,0.729730,0.712788,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.673469,0.666100,0.673469,0.661981,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [103]:
df_D.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.734694,0.734890,0.734694,0.717321
0.3,0.711712,0.704365,0.711712,0.686934
0.4,0.700680,0.700635,0.700680,0.668522
0.5,0.682927,0.674040,0.682927,0.668118


### Overall_rating_V2 `(Strategy)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [104]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'A/R_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_E = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_E

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.696000,0.672499,0.696000,0.679375,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.640000,0.647873,0.640000,0.611568,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
2,0.5,101,0.720000,0.697507,0.720000,0.698132,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.690000,0.667301,0.690000,0.672423,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.720000,0.717239,0.720000,0.702889,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.690000,0.657939,0.690000,0.652334,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.680000,0.658534,0.680000,0.654985,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.693333,0.673778,0.693333,0.670643,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
8,0.3,101,0.746667,0.719600,0.746667,0.724028,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
9,0.2,7,0.700000,0.731572,0.700000,0.673331,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."


In [105]:
df_E.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.793333,0.803819,0.793333,0.777139
0.3,0.706667,0.683971,0.706667,0.683219
0.4,0.700000,0.680826,0.700000,0.675882
0.5,0.685333,0.672627,0.685333,0.663025


### Overall_rating_V2 `(Strategy)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [106]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'S/I_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]


oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_F = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_F

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.768000,0.753487,0.768000,0.757788,"{'metric': 'manhattan', 'n_neighbors': 11, 'we..."
1,0.5,69,0.712000,0.698907,0.712000,0.680318,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.696000,0.674595,0.696000,0.680925,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.720000,0.721428,0.720000,0.710227,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.770000,0.753088,0.770000,0.748401,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.690000,0.670033,0.690000,0.658681,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.720000,0.684616,0.720000,0.685026,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
7,0.3,69,0.760000,0.750667,0.760000,0.744642,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.733333,0.740484,0.733333,0.731144,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.720000,0.736667,0.720000,0.676254,"{'metric': 'euclidean', 'n_neighbors': 11, 'we..."


In [107]:
df_F.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.793333,0.800976,0.793333,0.773880
0.3,0.737778,0.725255,0.737778,0.720271
0.4,0.726667,0.714850,0.726667,0.705770
0.5,0.725333,0.708996,0.725333,0.706344


### Overall_rating_V2 `(Strategy)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [108]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'Vi/Vb_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_G = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_G

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.688000,0.667548,0.688000,0.675288,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.456000,0.476399,0.456000,0.435578,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.768000,0.750742,0.768000,0.749695,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.720000,0.700593,0.720000,0.707020,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
4,0.4,69,0.660000,0.661944,0.660000,0.660125,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.770000,0.804154,0.770000,0.742851,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.666667,0.629121,0.666667,0.637553,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.733333,0.744411,0.733333,0.737337,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.813333,0.813845,0.813333,0.792680,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.640000,0.611319,0.640000,0.603333,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [109]:
df_G.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.773333,0.761490,0.773333,0.756121
0.3,0.737778,0.729126,0.737778,0.722523
0.4,0.716667,0.722230,0.716667,0.703332
0.5,0.637333,0.631563,0.637333,0.620187


### Overall_rating_V2 `(Strategy)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [110]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V2']]
X = training[['CGPA_Class',
              'Watch_Status_V2',
              'S/G_Score','Quiz','part_day_V2-1', 'part_day_V2-2', 'part_day_V2-3', 'part_day_V2-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_H = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_H

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.680000,0.669165,0.680000,0.672368,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.640000,0.645581,0.640000,0.624018,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
2,0.5,101,0.752000,0.740906,0.752000,0.729198,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.690000,0.671008,0.690000,0.677352,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
4,0.4,69,0.700000,0.698530,0.700000,0.692250,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
5,0.4,101,0.740000,0.755363,0.740000,0.709192,"{'metric': 'manhattan', 'n_neighbors': 11, 'we..."
6,0.3,7,0.720000,0.692194,0.720000,0.694213,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.746667,0.718768,0.746667,0.723742,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.800000,0.802596,0.800000,0.777396,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
9,0.2,7,0.640000,0.606517,0.640000,0.617537,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."


In [111]:
df_H.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.766667,0.748688,0.766667,0.747550
0.3,0.755556,0.737853,0.755556,0.731784
0.4,0.710000,0.708300,0.710000,0.692931
0.5,0.690667,0.685217,0.690667,0.675195


### Overall_rating_V3 `(MVC)` for `Active/Reflective` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [112]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'A/R_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_I = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_I

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.833333,0.821528,0.833333,0.813436,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.800000,0.822797,0.800000,0.788469,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.746667,0.756738,0.746667,0.742820,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.858333,0.864739,0.858333,0.840610,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.808333,0.826790,0.808333,0.788907,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
5,0.4,101,0.833333,0.820677,0.833333,0.818765,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
6,0.3,7,0.855556,0.840450,0.855556,0.840930,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
7,0.3,69,0.811111,0.810912,0.811111,0.797500,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.844444,0.833158,0.844444,0.826116,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.850000,0.828832,0.850000,0.835748,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."


In [113]:
df_I.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.833333,0.828191,0.833333,0.816320
0.3,0.837037,0.828173,0.837037,0.821515
0.4,0.833333,0.837402,0.833333,0.816094
0.5,0.793333,0.800354,0.793333,0.781575


### Overall_rating_V3 `(MVC)` for `Sensing/Intuitive` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [114]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'S/I_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_J = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_J

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.786667,0.780364,0.786667,0.783149,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.760000,0.753575,0.760000,0.735214,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.733333,0.706096,0.733333,0.716367,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.875000,0.883881,0.875000,0.858021,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
4,0.4,69,0.741667,0.725215,0.741667,0.717072,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.791667,0.775242,0.791667,0.760225,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
6,0.3,7,0.866667,0.866387,0.866667,0.844246,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.811111,0.797994,0.811111,0.786667,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
8,0.3,101,0.822222,0.854115,0.822222,0.793217,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
9,0.2,7,0.883333,0.870684,0.883333,0.861585,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."


In [115]:
df_J.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.861111,0.873366,0.861111,0.837730
0.3,0.833333,0.839499,0.833333,0.808043
0.4,0.802778,0.794779,0.802778,0.778439
0.5,0.760000,0.746679,0.760000,0.744910


### Overall_rating_V3 `(MVC)` for `Visual/Verbal` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [116]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'Vi/Vb_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_K = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_K

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.786667,0.780416,0.786667,0.777048,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
1,0.5,69,0.740000,0.732115,0.740000,0.731224,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.813333,0.809660,0.813333,0.798113,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.816667,0.817117,0.816667,0.800416,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
4,0.4,69,0.775000,0.769693,0.775000,0.752012,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
5,0.4,101,0.833333,0.822058,0.833333,0.814842,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
6,0.3,7,0.822222,0.824339,0.822222,0.817123,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.811111,0.796309,0.811111,0.800092,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
8,0.3,101,0.844444,0.874239,0.844444,0.803906,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
9,0.2,7,0.850000,0.844167,0.850000,0.844015,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."


In [117]:
df_K.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.850000,0.870895,0.850000,0.836221
0.3,0.825926,0.831629,0.825926,0.807040
0.4,0.808333,0.802956,0.808333,0.789090
0.5,0.780000,0.774064,0.780000,0.768795


### Overall_rating_V3 `(MVC)` for `Sequential/Global` learners (60 Train - 40 Test) | (70 Train - 30 Test) | (80 Train - 20 Test)

In [118]:
temp = []
rand_states = [7,69,101]
y = training[['Overall_Rating_V3']]
X = training[['CGPA_Class',
              'Watch_Status_V3',
              'S/G_Score','Quiz','part_day_V3-1', 'part_day_V3-2', 'part_day_V3-3', 'part_day_V3-4']]

oversample = RandomOverSampler(sampling_strategy='all')
X, y = oversample.fit_resample(X, y)


for i in [0.5,0.4,0.3,0.2]:
    for j in rand_states:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=j)
        param_grid = {'n_neighbors':[3,5,11],
                      'weights':['uniform','distance'],
                      'metric':['euclidean','manhattan']}

        model = GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1, cv=10, verbose=1)
        model.fit(X_train, y_train.values.ravel())
        pred = model.predict(X_test)
        
        acc = accuracy_score(y_test,pred)
        pre = precision_score(y_test, pred, average='weighted')
        recall = recall_score(y_test, pred, average='weighted')
        f1 = f1_score(y_test, pred, average='weighted')
        param = model.best_params_
        
        temp.append([i,j,acc,pre,recall,f1,param])
        
df_L = pd.DataFrame(columns=['Test Size','Random State','Accuracy','Precision','Recall','F1 Score','Best Param'],data=temp)
df_L

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 12 candidates, totalling 120 fits


,Test Size,Random State,Accuracy,Precision,Recall,F1 Score,Best Param
0,0.5,7,0.740000,0.728066,0.740000,0.724608,"{'metric': 'euclidean', 'n_neighbors': 11, 'we..."
1,0.5,69,0.753333,0.740730,0.753333,0.715982,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
2,0.5,101,0.760000,0.730665,0.760000,0.734507,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
3,0.4,7,0.850000,0.843634,0.850000,0.834978,"{'metric': 'euclidean', 'n_neighbors': 5, 'wei..."
4,0.4,69,0.766667,0.743465,0.766667,0.723939,"{'metric': 'manhattan', 'n_neighbors': 11, 'we..."
5,0.4,101,0.816667,0.811682,0.816667,0.782983,"{'metric': 'manhattan', 'n_neighbors': 3, 'wei..."
6,0.3,7,0.833333,0.819603,0.833333,0.816068,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."
7,0.3,69,0.800000,0.815030,0.800000,0.767361,"{'metric': 'manhattan', 'n_neighbors': 11, 'we..."
8,0.3,101,0.844444,0.838683,0.844444,0.826778,"{'metric': 'manhattan', 'n_neighbors': 5, 'wei..."
9,0.2,7,0.866667,0.857540,0.866667,0.846502,"{'metric': 'euclidean', 'n_neighbors': 3, 'wei..."


In [119]:
df_L.groupby('Test Size')[['Accuracy','Precision','Recall','F1 Score']].mean()

,Accuracy,Precision,Recall,F1 Score
Test Size,,,,
0.2,0.827778,0.825164,0.827778,0.811251
0.3,0.825926,0.824439,0.825926,0.803402
0.4,0.811111,0.799593,0.811111,0.780633
0.5,0.751111,0.733154,0.751111,0.725032
